# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from datasets import load_dataset

# https://huggingface.co/datasets/dair-ai/emotion

from datasets import load_dataset

dataset = load_dataset("dair-ai/emotion" )

splits = ["train", "test"]

# View the dataset characteristics
dataset["train"]

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 16000
})

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Let's use a lambda function to tokenize all the examples
# Setting max length to 300 as that is what the max length appears
# to be in the dataset
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["text"], truncation=True, padding="max_length", max_length=300), batched=True
    )

# Inspect the available columns in the dataset
tokenized_dataset["train"]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 16000
})

In [3]:
# View the dataset characteristics
dataset["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 16000
})

In [4]:
dataset["test"]

Dataset({
    features: ['text', 'label'],
    num_rows: 2000
})

In [5]:
dataset["train"][0]

{'text': 'i didnt feel humiliated', 'label': 0}

In [6]:
# Replace <MASK> with the code to unfreeze all the model parameters

from transformers import AutoModelForSequenceClassification
id2Label= {0 :'anger', 1 : 'fear', 
    2 : 'joy', 3 : 'love', 
    4 : 'sadness', 5 : 'surprise'}
label2Id = {'anger':0, 'fear': 1, 
   'joy':2, 'love': 3, 
    'sadness': 4, 'surprise': 5}
model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=6,
    id2label=id2Label,
    label2id=label2Id,
)
model.config.pad_token_id = tokenizer.pad_token_id

# Unfreeze all the model parameters.
# Hint: Check the documentation at https://huggingface.co/transformers/v4.2.2/training.html
for param in model.parameters():
    param.requires_grad = True

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/emotion_rating_initial",
        # Set the learning rate
        learning_rate=0.00002,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [8]:
# Show the performance of the model on the test set
trainer.evaluate()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 9.250245094299316,
 'eval_accuracy': 0.033,
 'eval_runtime': 36.4075,
 'eval_samples_per_second': 54.934,
 'eval_steps_per_second': 3.433}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [9]:
from peft import LoraConfig, TaskType
config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    lora_dropout=0.2
)

In [10]:
from peft import get_peft_model
lora_model = get_peft_model(model, config)

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [11]:
lora_model.config.pad_token_id = tokenizer.pad_token_id

In [12]:
lora_model.print_trainable_parameters()

trainable params: 304,128 || all params: 124,743,936 || trainable%: 0.2438018309763771


In [13]:
# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/emotion_rating_lora_model",
        # Set the learning rate
        learning_rate=0.00002,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.672900,1.595787,0.349000
2,1.611400,1.561208,0.374000


TrainOutput(global_step=2000, training_loss=2.401436279296875, metrics={'train_runtime': 1490.6889, 'train_samples_per_second': 21.467, 'train_steps_per_second': 1.342, 'total_flos': 4916743372800000.0, 'train_loss': 2.401436279296875, 'epoch': 2.0})

In [15]:
lora_model.save_pretrained('model/lora_model')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [16]:
from peft import AutoPeftModelForSequenceClassification
inference_model = AutoPeftModelForSequenceClassification.from_pretrained(
    "model/lora_model",
    num_labels=6,
    id2label=id2Label,
    label2id=label2Id,
)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
inference_model.config.pad_token_id = inference_model.config.eos_token_id

In [18]:
# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=inference_model,
    args=TrainingArguments(
        output_dir="./data/yelp_ratings_lora_finetuned",
        # Set the learning rate
        learning_rate=0.00002,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [19]:
trainer.evaluate()

{'eval_loss': 1.5612080097198486,
 'eval_accuracy': 0.374,
 'eval_runtime': 39.488,
 'eval_samples_per_second': 50.648,
 'eval_steps_per_second': 3.166}